In [4]:
import pandas as pd
import numpy as np

In [5]:
# FONTE: https://portal.inmet.gov.br/dadoshistoricos   BENTO GONCALVES - RS

# carregando varios arquivos csv e adicionando uma coluna com o nome do arquivo
arquivos_csv = []

for i in range(2007, 2024): 
    arquivo = f'../Data/input/Meteorologia/Bento Goncalves/INMET_S_RS_A840_BENTO GONCALVES_01-01-{i}_A_31-12-{i}.csv'
    arquivos_csv.append(arquivo)
    print(arquivo)


df = pd.concat([pd.read_csv(f,sep=';', skiprows=9, encoding='latin1', header=None, usecols=lambda column : column != 19).assign(arquivo=f) for f in arquivos_csv ])


../Data/input/Meteorologia/Bento Goncalves/INMET_S_RS_A840_BENTO GONCALVES_01-01-2007_A_31-12-2007.csv
../Data/input/Meteorologia/Bento Goncalves/INMET_S_RS_A840_BENTO GONCALVES_01-01-2008_A_31-12-2008.csv
../Data/input/Meteorologia/Bento Goncalves/INMET_S_RS_A840_BENTO GONCALVES_01-01-2009_A_31-12-2009.csv
../Data/input/Meteorologia/Bento Goncalves/INMET_S_RS_A840_BENTO GONCALVES_01-01-2010_A_31-12-2010.csv
../Data/input/Meteorologia/Bento Goncalves/INMET_S_RS_A840_BENTO GONCALVES_01-01-2011_A_31-12-2011.csv
../Data/input/Meteorologia/Bento Goncalves/INMET_S_RS_A840_BENTO GONCALVES_01-01-2012_A_31-12-2012.csv
../Data/input/Meteorologia/Bento Goncalves/INMET_S_RS_A840_BENTO GONCALVES_01-01-2013_A_31-12-2013.csv
../Data/input/Meteorologia/Bento Goncalves/INMET_S_RS_A840_BENTO GONCALVES_01-01-2014_A_31-12-2014.csv
../Data/input/Meteorologia/Bento Goncalves/INMET_S_RS_A840_BENTO GONCALVES_01-01-2015_A_31-12-2015.csv
../Data/input/Meteorologia/Bento Goncalves/INMET_S_RS_A840_BENTO GONCALVE

In [6]:
# Definindo o nome das colunas
df.columns = ['Data', 'Hora', 'Precipitacao', 'PressaoAtmEstacao', 'PressaoAtmMax', 'PressaoAtmMin', 'RadiacaoGlobal', 'TempAr', 'TempPontoOrvalho', 'TempMax', 'TempMin', 'TempOrvalhoMax', 'TempOrvalhoMin', 'UmidadeRelMax', 'UmidadeRelMin', 'UmidadeRel', 'VentoDir', 'VentoRajadaMax', 'VentoVelocidade', 'Arquivo']

In [7]:
df['Data'] = pd.to_datetime(df['Data'].replace('/', '-', regex=True))

In [8]:
columns = ['Precipitacao', 'PressaoAtmEstacao', 'PressaoAtmMax', 'PressaoAtmMin', 'RadiacaoGlobal', 'TempAr', 'TempPontoOrvalho', 'TempMax', 'TempMin', 'TempOrvalhoMax', 'TempOrvalhoMin', 'UmidadeRelMax', 'UmidadeRelMin', 'UmidadeRel', 'VentoDir', 'VentoRajadaMax', 'VentoVelocidade']
for column in columns:    
    df[column] = pd.to_numeric(df[column].replace(',', '.', regex=True).astype(float), errors='coerce')
    df[column] = df[column].replace(-9999, np.nan)
    df[column] = df[column].replace(0, np.nan)

In [9]:
df['Ano'] = df['Data'].dt.year

# pegar semestre
df['Semestre'] = np.where(df['Data'].dt.month < 7, 1, 2)

In [10]:
# agrupar por ano, semestre e arquivo e calcular a média, min e max
df_media = df.groupby(['Ano','Semestre', 'Arquivo'])[columns].agg(['median', 'mean', 'min', 'max']).reset_index()


In [11]:
# salvando o arquivo
df_media.to_csv('../Data/output/Meteorologia_Semestre.csv', index=False)

In [12]:
# agrupar por ano, semestre e arquivo e calcular a média, min e max
df_media_ano = df.drop(columns='Semestre').groupby(['Ano', 'Arquivo'])[columns].median().reset_index()


In [13]:
# salvando o arquivo
df_media_ano.to_csv('../Data/output/Meteorologia_Ano_mediana.csv', index=False)